In [1]:
## first compressing the SCC data? then label it from 1 to len(data) as n distinct labels (corrosponding to distinct nodes)
## no need to reverse the (transposed data) as long as u label it from n to 1 from the first node (not efficient as to manipulate the data to much)
## graph: modified to have 3 extra position in each node [T/F(as explored or not,leader_s(group_index),f(i)=t(as time been explored))]
## example node [2(start_of_edge), 5(end_of_edge), T/F, leader_s(int), t(list[0])]

## if data is not connected numbers, fill in the gap!!!

In [2]:
import copy

In [3]:
def load_graph(file_name):
    with open(file_name) as f:
        data_set = []
        for ln in f:
            line = ln.split()
            if line:
                a = [int(x) for x in line]
                data_set.append(a)
    return data_set

# file_name = 'directed graph_test.txt'    ## for input the data file and make a test_copy of it

In [4]:
def list_of_zeros(n):  ## function used to initialize reverse_scc-1-n sorted
    result = [[0,0]]*n
    return result

In [5]:
def reverse_scc_graph_sorted(graph):  ## function to reverse scc graph, 1-n sorted
    rev_graph = list_of_zeros(len(graph))
    for index in range(len(graph)):
        node = graph[index]
        rev_graph[index] = node[::-1]
    rev_graph.sort()
        
    return rev_graph        

In [6]:
def compress_graph(a):  ## compress data with distinct nodes
    b = [[0,0]]
    for index in range(len(a)):
        if b[-1][0]==a[index][0]:
            b[-1].append(a[index][1])
        else:
            b.append(a[index])
    b.pop(0)
    return b
    

In [24]:
def fill_blank(graph):
    for index in range(graph[-1][0]):
        if not graph[index][0] == index+1:
            graph.insert(index,[index+1])
    return graph

In [25]:
def add_init_flag(graph,t_graph,checker):
    flag = False    
    for x in range(0,len(graph)):
        graph[x].append(flag)  # add an initial flag of false(as unexplored) AT THE END of each vertex
        graph[x].append(0)  # position for leader_s
        graph[x].append(0)  # postiion for f(i) := t
        if checker==1:
            graph[x] = [len(graph)-x] + graph[x]  # added a label for 1-n distinct nodes
        elif checker== 2:
            graph[x] = [t_graph[x][-1]] + graph[x]  # added a label for 1-n distinct nodes with t
   
    return graph

In [26]:
def check_flag(vertex):  # check flag of one vertex, if unexplored(false flag), then mark it as explored(flag==true) and return true
    if vertex[-3] is False:  # flag position at -3       
        return 1
    else:
        return 0    

In [27]:
class Stack(list):
    def __init__(self):
        self = []
    
    def push_s(self, item):
        self.append(item)
    
    def pop_s(self):
        return self.pop()
    
    def is_empty(self):
        return len(self)==0
    
    def peek(self):
        if not self.is_empty():
            return self[-1]
    
    def get_stack(self):
        return self

In [11]:
def prep(graph,t_graph,checker):  ## load in compressed scc graph data for labeling output
    graph_copy = graph.copy()
    
    stack_of_vertex = Stack()  # initiate an empty stack
    if checker==1:
        graph_rev = reverse_scc_graph_sorted(graph_copy)
    else:
        graph_rev = graph_copy
        
    graph_compressed = compress_graph(graph_rev)
    graph_fill = fill_blank(graph_compressed)       
    graph_flag = add_init_flag(graph_fill,t_graph,checker)

    
    return graph_flag, stack_of_vertex 

In [19]:
def find_scc_1(graph):
    ## graph as input is the original graph
    original_graph = graph.copy()
    graph_flag, stack_of_vertex_1 = prep(graph,[],1)
    
    ## first pass loop to determin t (magic order)
    t = [0]  # number of nodes processed so far
    print(graph_flag)

    
    for index in range(len(graph_flag)):
        if check_flag(graph_flag[index]):
            depth_first_search_first(graph_flag,index+1,stack_of_vertex_1,t)
            
    return graph_flag
    
#     ## second pass to get scc
    
#     leader_s = 0  # used in second pass to record label of scc
#     leader_recorder = []
# #     scc_lib = {}
#     scc_list = []
#     scc_size = 0
    
#     original_t_flag, stack_of_vertex_2 = prep(original_graph,graph_flag,2)
#     original_flag = original_t_flag
#     remove_zero = []
    
    
#     for index in range(len(original_t_flag)):
#         if original_t_flag[index][0] != 0:
# #             print("there is a zero", index)
#             remove_zero.append(original_t_flag[index])
    
#     remove_zero.sort()   
#     original_t_flag = remove_zero[::-1]      

    
#     for index in range(len(original_t_flag)):
#         if check_flag(original_t_flag[index]):
#             scc_size = 0
#             leader_s = original_t_flag[index][0]
# #             scc_lib[lead_s] = scc_size
#             leader_recorder.append(leader_s)
#             scc_size = depth_first_search_second(original_t_flag,index+1,stack_of_vertex_2,leader_s,original_flag,scc_size)
        
#             scc_list.append(scc_size)
            
    
    
        
    
#     return leader_recorder, scc_list

In [20]:
def find_scc_2(graph):
    ## graph as input is the original graph
    original_graph = graph.copy()
    graph_flag, stack_of_vertex_1 = prep(graph,[],1)
    
    ## first pass loop to determin t (magic order)
    t = [0]  # number of nodes processed so far

    
    for index in range(len(graph_flag)):
        if check_flag(graph_flag[index]):
            depth_first_search_first(graph_flag,index+1,stack_of_vertex_1,t)
    
    ## second pass to get scc
    
    leader_s = 0  # used in second pass to record label of scc
    leader_recorder = []
#     scc_lib = {}
    scc_list = []
    scc_size = 0
    
    original_t_flag, stack_of_vertex_2 = prep(original_graph,graph_flag,2)
    original_flag = original_t_flag
    remove_zero = []
    
    
    for index in range(len(original_t_flag)):
        if original_t_flag[index][0] != 0:
#             print("there is a zero", index)
            remove_zero.append(original_t_flag[index])
    
    remove_zero.sort()   
    original_t_flag = remove_zero[::-1]      

    
    for index in range(len(original_t_flag)):
        if check_flag(original_t_flag[index]):
            scc_size = 0
            leader_s = original_t_flag[index][0]
#             scc_lib[lead_s] = scc_size
            leader_recorder.append(leader_s)
            scc_size = depth_first_search_second(original_t_flag,index+1,stack_of_vertex_2,leader_s,original_flag,scc_size)
        
            scc_list.append(scc_size)
            
    
    
        
    
    return leader_recorder, scc_list

In [21]:
def depth_first_search_first(graph_flag,s,stack,t):  # s is the index of that vertex of interest (s-1 for index in graph)
    
    stack_of_vertex = stack

    graph_flag[s-1][-3] = True  # mark s as explored    

    # initialize Q with s
    stack_of_vertex.push_s(graph_flag[s-1][0])
    

    if not stack_of_vertex.is_empty():        
        m = stack_of_vertex[-1]
#         print("current m: ", m) # for debug
        length = len(graph_flag)
        node_m = graph_flag[length-m]
        
        if len(node_m)>5:
            for index in range(2,len(node_m)-3):
                n = node_m[index]
#                 print("n:" ,n)
                if check_flag(graph_flag[n-1]):
                    print("run recursion")
                    depth_first_search_first(graph_flag,n,stack_of_vertex,t)              
            

            t[0] = t[0] + 1
            graph_flag[length-m][-1] = t[0]

            stack_of_vertex.pop()
     
    return  
            

In [22]:
def depth_first_search_second(original_t_flag,s,stack,leader_s,original_flag,scc_size):  # s is the index of that vertex of interest (s-1 for index in graph)
    
    stack_of_vertex = stack
#     print(original_t_flag)
#     print(original_flag)

    original_t_flag[s-1][-3] = True  # mark s as explored
    i = original_t_flag[s-1][1]
    original_flag[i-1][-3] = True
    original_flag[i-1][-2] = leader_s  # assign leader_s
    scc_size += 1
    

    # initialize Q with s
    stack_of_vertex.push_s(original_t_flag[s-1][0])
    

    if not stack_of_vertex.is_empty():        
        m = stack_of_vertex[-1]
        i = original_t_flag[s-1][1]
        length = len(original_t_flag)
        node_m = original_t_flag[length-m]
    
        for index in range(2,len(node_m)-3):
            n = node_m[index]
#             print(original_flag[n-1])
            if check_flag(original_flag[n-1]):
                n_t = length + 1 - original_flag[n-1][0]
                scc_size = depth_first_search_second(original_t_flag,n_t,stack_of_vertex,leader_s,original_flag,scc_size)         
       
        stack_of_vertex.pop()
         
    return scc_size 

In [16]:
# importing the sys module 
import sys 
  
# the setrecursionlimit function is 
# used to modify the default recursion 
# limit set by python. Using this,  
# we can increase the recursion limit 
# to satisfy our needs 
  
sys.setrecursionlimit(10**6) 

In [17]:
# def check_dup(graph):
#     result = 0
#     for index in range(len(graph)):
#         if graph[index][0] == graph[index][1]:
#             result += 1
#     return result
            

In [19]:
%%time
file_name = 'SCC_no_duplicate.txt'
graph = load_graph(file_name)
graph_copy = graph.copy()
graph_flag, stack_of_vertex_1 = prep(graph_copy,[],1)

print(len(graph_copy))
print(len(graph_flag))


5105042
875709
CPU times: user 28.5 s, sys: 226 ms, total: 28.7 s
Wall time: 28.6 s


In [29]:
%%time
file_name = 'SCC_test_example (disconnect).txt'
graph = load_graph(file_name)
graph_copy = graph.copy()

print(len(graph_copy))
graph_t = find_scc_1(graph_copy)

# print(recorder)
# print(scc_list)
print(graph_t)


    

11
[[15, 1, 5, False, 0, 0], [14, 2, False, 0, 0], [13, 3, 1, False, 0, 0], [12, 4, False, 0, 0], [11, 5, 3, 6, False, 0, 0], [10, 6, 9, False, 0, 0], [9, 7, 6, False, 0, 0], [8, 8, False, 0, 0], [7, 9, 7, 10, False, 0, 0], [6, 10, 11, False, 0, 0], [5, 11, 15, False, 0, 0], [4, 12, False, 0, 0], [3, 13, False, 0, 0], [2, 14, False, 0, 0], [1, 15, 10, False, 0, 0]]
run recursion
run recursion
run recursion
run recursion
run recursion
run recursion
run recursion
run recursion
[[15, 1, 5, True, 0, 9], [14, 2, True, 0, 0], [13, 3, 1, True, 0, 1], [12, 4, True, 0, 0], [11, 5, 3, 6, True, 0, 8], [10, 6, 9, True, 0, 7], [9, 7, 6, True, 0, 2], [8, 8, True, 0, 0], [7, 9, 7, 10, True, 0, 6], [6, 10, 11, True, 0, 5], [5, 11, 15, True, 0, 4], [4, 12, True, 0, 0], [3, 13, True, 0, 0], [2, 14, True, 0, 0], [1, 15, 10, True, 0, 3]]
CPU times: user 1.09 ms, sys: 105 µs, total: 1.2 ms
Wall time: 986 µs


In [19]:
def find_jumped_index(compressed):
    result = []
    for index in range(len(compressed)):        
        if index+1 != compressed[index][0]:
            result.append(index)
    return result

In [20]:
result = find_jumped_index(compressed)
print(len(result))

NameError: name 'compressed' is not defined

In [141]:
a = [1,2,3]
b = [2]
b = b+a
print(b)

[2, 1, 2, 3]


In [17]:
a = [1,2,3]
flag = False
a.append(flag)
a.append(0)
a.append(0)
print(a)

[1, 2, 3, False, 0, 0]


In [22]:
a = [0]
b = [1,2,3]
b[-1] = int(a[0])
print(b)

[1, 2, 0]


In [ ]:
a = [[9,9],[8,7],[8,6],[7,2],[6,4]]

In [33]:
def compress_graph(a):
    b = [[0,0]]
    for index in range(len(a)):
        if b[-1][0]==a[index][0]:
            b[-1].append(a[index][1])
        else:
            b.append(a[index])
    b.pop(0)
    return b
    

[[1, 0, 2, 3], [2, 4], [3, 6]]


0


In [220]:
a = [[9,9],[8,7],[8,6],[7,2],[6,4]]
b = 3
a.insert(2,[b])
print(a)

[[9, 9], [8, 7], [3], [8, 6], [7, 2], [6, 4]]


In [417]:
a = [[9,9],[8,7],[8,6],[7,2],[6,4]]
b = []
for index in range(len(a)):
    if a[index][0]!=8:
        b.append(a[index])

a = b
print(b)
print(a)


[[9, 9], [7, 2], [6, 4]]
[[9, 9], [7, 2], [6, 4]]


In [413]:
for i in range(3):
    print ('f')

f
f
f


In [441]:
a = [[9, 1, 5, False, 0, 0], [1, 3, 1, False, 0, 0], [8, 5, 3, 6, False, 0, 0], [7, 6, 9, False, 0, 0], [2, 7, 6, False, 0, 0], [6, 9, 7, 10, False, 0, 0], [5, 10, 11, False, 0, 0], [4, 11, 15, False, 0, 0], [3, 15, 10, False, 0, 0]]
a.sort()
b = a[::-1]
print(b)


[[9, 1, 5, False, 0, 0], [8, 5, 3, 6, False, 0, 0], [7, 6, 9, False, 0, 0], [6, 9, 7, 10, False, 0, 0], [5, 10, 11, False, 0, 0], [4, 11, 15, False, 0, 0], [3, 15, 10, False, 0, 0], [2, 7, 6, False, 0, 0], [1, 3, 1, False, 0, 0]]


In [472]:
lib = {}
lead_s = 5
lib[lead_s] = 4
print(lib)

{5: 4}


In [522]:
a = {1:2}
for i in range(3):
    a[i] = i
    
print(a)

{1: 1, 0: 0, 2: 2}


In [30]:
l = [0, 10, 20, 30, 40, 50, 60]
print(l)
# [0, 10, 20, 30, 40, 50, 60]

[0, 10, 20, 30, 40, 50, 60]


In [31]:
sl = slice(2, 5, 2)
print(sl)
# slice(2, 5, 2)

print(type(sl))
# <class 'slice'>

print(l[sl])
# [20, 40]

slice(2, 5, 2)
<class 'slice'>
[20, 40]


In [32]:
sl = slice(2, 5)
print(sl)
# slice(2, 5, None)

print(l[sl])
# [20, 30, 40]

slice(2, 5, None)
[20, 30, 40]


In [34]:
a = ([1,2],[2,3])
b = a[0]
print(b)

[1, 2]


In [35]:
a = True
if a:
    b = 3
    a = False
print(b)

3
